In [ ]:
import music21

music21.environment.set('musicxmlPath', '/Applications/MuseScore 4.app')

In [ ]:
from music21 import converter, tempo

# Load the MIDI file
score = converter.parse(
    "/Users/fernando/dev/upf/cmc24/gdrm/dataset/drummer1/session1/4_jazz-funk_116_beat_4-4.mid",
    format="midi",
    quantizePost=False,
)

In [ ]:
score.show("xml")

In [ ]:


# Get tempo markings from the score
tempo_markings = score.metronomeMarkBoundaries()

# Display BPM
for start, end, metronomeMark in tempo_markings:
    if isinstance(metronomeMark, tempo.MetronomeMark):
        print(f"Tempo: {metronomeMark.number} BPM (from offset {start} to {end})")

In [ ]:
tempo_markings

In [ ]:
midi_path = "music21_output.mid"
score.write('midi', fp=midi_path)

print(f"MIDI file saved to {midi_path}")

In [ ]:
score.__dir__()

In [ ]:
converter.subConverters.ConverterJSON

In [ ]:
for part in score.parts:
    print(part)
    for note in part.flat.notes:
        print(note)

In [ ]:
note

In [ ]:
import muspy

music = muspy.from_music21_score(score)

In [ ]:
score.show("text")

In [ ]:
music_dict = music.to_ordered_dict()

In [ ]:
dict(dict(music_dict)["barlines"][0])

In [ ]:
from music21 import stream

my_dict = {}

# Iterate over each Part in the Score
for p in score.parts:
    # Iterate over the Measure objects in each Part
    for meas in p.getElementsByClass(stream.Measure):
        offset = meas.offset  # the offset of the measure in the Part
        measure_number = meas.measureNumber
        
        # Gather the notes (and rests) in that measure
        # (or use notesAndRests, or getElementsByClass('Note') if you only want notes, etc.)
        notes_list = list(meas.notesAndRests)
        
        # You can store anything else you want, like key signatures, clefs, etc.:
        # key_signature = meas.keySignature
        # clef = meas.clef
        # time_signature = meas.timeSignature
        
        # Build up the dictionary entry
        my_dict[offset] = {
            'measure_number': measure_number,
            'notes': notes_list,
            # 'key_signature': key_signature,
            # 'clef': clef,
            # 'time_signature': time_signature,
        }

# Now my_dict has your measure offsets as keys, and each value holds measure metadata.


In [ ]:
from music21 import stream

In [ ]:
my_notes_info = []
my_just_notes = []
my_pitches = []

for part in score.parts:
    for measure in part.getElementsByClass(stream.Measure):
        # measureOffset is measure.offset -- offset in the Part
        measureOffset = measure.offset
        measureNumber = measure.measureNumber
        
        for n in measure.notesAndRests:
            # offset in the measure:
            noteLocalOffset = n.offset  
            # absolute offset in the entire Score:
            noteGlobalOffset = n.getOffsetInHierarchy(score)

            my_notes_info.append({
                'measureNumber': measureNumber,
                'measureOffset': measureOffset,
                'noteLocalOffset': noteLocalOffset,
                'noteGlobalOffset': noteGlobalOffset,
                'element': n,
                "measureOffsetFloat": float(measureOffset),
                "noteLocalOffsetFloat": float(noteLocalOffset),
                "noteGlobalOffsetFloat": float(noteGlobalOffset),
            })

        for n in measure.notes:
            my_just_notes.append(n)

        for n in measure.pitches:
            my_pitches.append(n)

# Now `my_notes_info` is a list of dicts describing each note/rest’s offsets

In [ ]:
score.parts[0].__dir__()

In [ ]:
measure.__dir__()

In [ ]:
my_notes_info

In [ ]:
# imoprt Fraction
from fractions import Fraction

In [ ]:
float(Fraction(817, 240))

In [ ]:
my_notes_info[17]

In [ ]:
269 + (4*240)

In [ ]:
import numpy as np
sixteenth_divs = np.linspace(0, 4, 32, endpoint=False)
eighth_triplet_divs = np.linspace(0, 4, 24, endpoint=False)
total_divs = set(sixteenth_divs).union(set(eighth_triplet_divs))
total_divs = sorted(list(total_divs))

total_divs.append(4.0)
print(total_divs)
print(len(total_divs))


In [ ]:
def closest_value(target, values=tuple(total_divs)):
    """
    Returns the closest value in 'values' to the given 'target'.
    """
    return min(values, key=lambda x: abs(x - target))

# Example usage:
print(closest_value(0.1))     # Should return 0.125 (since it's closest to 0.1)
print(closest_value(0.2))     # Could return 0.166666..., as it's closer to 0.2 than 0.125
print(closest_value(0.3))     # Should return 0.25

In [ ]:
closest_value(3.99)

In [ ]:
# for note in my_notes_info:
#     note["quantized_value"] = closest_value(note["noteLocalOffsetFloat"])
# print("Done")

In [ ]:
my_notes_info

In [ ]:
from collections import defaultdict

In [ ]:
bars = defaultdict(list)
for note in my_notes_info:
    quantized_value = closest_value(note["noteLocalOffsetFloat"])
    if quantized_value == 4.0:
        print(f"Snapping forward note: {note}")
        bars["measureNumber" + str(note["measureNumber"] + 1)].append({
            "element": note["element"],
            "quantized_value": 0.0
        })
    else:
        bars["measureNumber" + str(note["measureNumber"])].append({
            "element": note["element"],
            "quantized_value": quantized_value,
        })

In [ ]:
import json

In [ ]:
dict(bars)

In [ ]:
dict(bars).keys()

In [ ]:
my_pitches

In [ ]:
my_just_notes

In [ ]:
my_just_notes = []

for part in score.parts:
    for measure in part.getElementsByClass(stream.Measure):
        # measureOffset is measure.offset -- offset in the Part
        measureOffset = measure.offset
        measureNumber = measure.measureNumber
        
        for n in measure.notes:
            # offset in the measure:
            noteLocalOffset = n.offset  
            # absolute offset in the entire Score:
            noteGlobalOffset = n.getOffsetInHierarchy(score)

            my_just_notes.append({
                'measureNumber': measureNumber,
                'measureOffset': measureOffset,
                'noteLocalOffset': noteLocalOffset,
                'noteGlobalOffset': noteGlobalOffset,
                'element': n,
                "measureOffsetFloat": float(measureOffset),
                "noteLocalOffsetFloat": float(noteLocalOffset),
                "noteGlobalOffsetFloat": float(noteGlobalOffset),
            })

        # for n in measure.notes:
        #     my_just_notes.append(n)

        # for n in measure.pitches:
        #     my_pitches.append(n)

# Now `my_notes_info` is a list of dicts describing each note/rest’s offsets

In [ ]:
len(my_just_notes)

In [ ]:
len(my_notes_info)

In [ ]:
from music21 import stream, note, instrument, tempo, meter

# Create an empty Score
score = stream.Score()

# Create a Part for the drum track
drum_part = stream.Part()
drum_part.id = 'Drums'

# Insert a percussive instrument (this helps some MIDI players recognize it as percussion)
drum_part.insert(0, instrument.UnpitchedPercussion())

# Set a tempo (optional)
drum_part.insert(0, tempo.MetronomeMark(number=120))  # 120 BPM

# Set time signature (optional)
drum_part.append(meter.TimeSignature('4/4'))

#------------------------------------------------------------------------------
# Add some drum notes explicitly
# By default in General MIDI, channel 10 is often the drum channel,
# and different pitch values correspond to different drum sounds:
#   36 = Bass Drum 1
#   38 = Acoustic Snare
#   42 = Closed Hi-Hat
#   46 = Open Hi-Hat
# etc.
#------------------------------------------------------------------------------

# Measure 1 (4 quarter notes)
drum_part.append(note.Note(36, quarterLength=1))  # Kick
drum_part.append(note.Note(38, quarterLength=1))  # Snare
drum_part.append(note.Note(36, quarterLength=1))  # Kick
drum_part.append(note.Note(38, quarterLength=1))  # Snare

# Measure 2 (4 quarter notes but let's mix it up a bit)
drum_part.append(note.Note(36, quarterLength=0.5)) # Kick on eighth note
drum_part.append(note.Note(36, quarterLength=0.5)) # Kick again
drum_part.append(note.Note(38, quarterLength=1))   # Snare
drum_part.append(note.Note(42, quarterLength=1))   # Closed Hi-hat

# Repeat (just as an example)
drum_part.append(note.Note(36, quarterLength=1))   # Kick
drum_part.append(note.Note(38, quarterLength=1))   # Snare
drum_part.append(note.Note(46, quarterLength=2))   # Open Hi-hat (2 beats)

# Add the drum Part into the Score
score.insert(0, drum_part)

# Show the textual representation of the score in the console
print(score.show('text'))

# If you want to see the staff (in a music notation program) and also
# hear the MIDI, you can do:
# score.show()        # Tries to open your default music notation software
score.show('xml')  # Plays back using your default MIDI player

# You can also write the score to a MIDI file:
# score.write('midi', fp='drum_track.mid')

In [ ]:
from music21 import stream, note, chord, instrument, tempo, meter

# Create an empty Score
score = stream.Score()

# Create a Part for the drum track
drum_part = stream.Part()
drum_part.id = 'Drums'

# Insert a percussive instrument (this helps MIDI players recognize it as a drum part)
drum_part.insert(0, instrument.UnpitchedPercussion())

# Set a tempo (120 BPM) and a 4/4 time signature
drum_part.insert(0, tempo.MetronomeMark(number=120))
drum_part.append(meter.TimeSignature('4/4'))

# Define one measure of a classic rock beat, broken into 8 eighth-notes:
# Beat division:
#   1    &    2    &    3    &    4    &
#   Kick/HiHat   HiHat   Snare/HiHat   HiHat   Kick/HiHat   HiHat   Snare/HiHat   HiHat

rock_measure = [
    chord.Chord([36, 42], quarterLength=0.5),  # Kick + Closed Hi-Hat
    chord.Chord([42],      quarterLength=0.5), # Closed Hi-Hat
    chord.Chord([38, 42], quarterLength=0.5),  # Snare + Closed Hi-Hat
    chord.Chord([42],      quarterLength=0.5), # Closed Hi-Hat
    chord.Chord([36, 42], quarterLength=0.5),  # Kick + Closed Hi-Hat
    chord.Chord([42],      quarterLength=0.5), # Closed Hi-Hat
    chord.Chord([38, 42], quarterLength=0.5),  # Snare + Closed Hi-Hat
    chord.Chord([42],      quarterLength=0.5), # Closed Hi-Hat
]

# Append two measures of this pattern:
drum_part.append(rock_measure)
# drum_part.append(rock_measure)

# Add the drum part to the score
score.insert(0, drum_part)

# Display a text summary of the score
print(score.show('text'))

# If you want to see the traditional notation (and have MuseScore or Finale, etc. installed):
score.show("xml")

# If you want to hear it:
# score.show('midi')      # Plays the MIDI with your default MIDI player
# score.write('midi', fp='rock_beat.mid')  # Saves it to a MIDI file

In [ ]:
from music21 import stream, note, instrument, tempo, meter
from copy import deepcopy
# Create an empty Score
score = stream.Score()

# Create a Part for the drum track
drum_part = stream.Part()
drum_part.id = 'Drums'

# Insert a percussive instrument (helps with GM MIDI channel 10 recognition)
drum_part.insert(0, instrument.UnpitchedPercussion())

# Set a tempo (120 BPM) and a 4/4 time signature
drum_part.insert(0, tempo.MetronomeMark(number=120))
drum_part.append(meter.TimeSignature('4/4'))

# Create a Measure
m1 = stream.Measure()
m1.number = 1  # measure number (optional but good practice)

# QuarterLength of an eighth note is 0.5
# We will place notes at offsets in the measure (0, 0.5, 1.0, 1.5, etc.)

#
#  1     &     2     &     3     &     4     &
#  offset=0   0.5    1.0   1.5    2.0   2.5    3.0   3.5
#

# Offsets (in quarterLength units) for a 4/4 bar subdivided into eighths
offsets = [i*0.5 for i in range(8)]  # [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]

# Basic rock pattern:
#  - Kick (36) on beats 1, 3  -> offsets 0, 2.0
#  - Snare (38) on beats 2, 4 -> offsets 1.0, 3.0
#  - Hi-hat (42) on every 8th note -> offsets 0,0.5,1.0,1.5,2.0,2.5,3.0,3.5

for off in offsets:
    # Hi-Hat always
    hat = note.Note(42, quarterLength=0.5)
    m1.insert(off, hat)
    
    # Kick on offset 0 and 2.0
    if off == 0 or off == 2.0:
        kick = note.Note(36, quarterLength=0.5)
        m1.insert(off, kick)
    
    # Snare on offset 1.0 and 3.0
    if off == 1.0 or off == 3.0:
        snare = note.Note(38, quarterLength=0.5)
        m1.insert(off, snare)

# Append measure 1 to the part
drum_part.append(m1)

# Make a second measure (exact same pattern but new measure) just by cloning
m2 = deepcopy(m1)
m2.number = 2
drum_part.append(m2)

# Add the drum part to the score
score.insert(0, drum_part)

# Show the textual representation
print(score.show('text'))

# For MIDI playback:
score.show('xml')
# Or write out to MIDI:
# score.write('midi', fp='rock_beat_individual_notes.mid')